In [ ]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [2]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.input.loaders.dfs import (
    store_entity_semantic_embeddings,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.question_gen.local_gen import LocalQuestionGen
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

/Users/zocho/Library/Caches/pypoetry/virtualenvs/graphfleet-bVb82vZ5-py3.12/lib/python3.12/site-packages/azure/search/documents/indexes/_generated/models/_models_py3.py:5644: SyntaxWarning: invalid escape sequence '\W'
  pattern: str = "\W+",
/Users/zocho/Library/Caches/pypoetry/virtualenvs/graphfleet-bVb82vZ5-py3.12/lib/python3.12/site-packages/azure/search/documents/indexes/_generated/models/_models_py3.py:5869: SyntaxWarning: invalid escape sequence '\W'
  pattern: str = "\W+",


## Local Search Example

Local search method generates answers by combining relevant data from the AI-extracted knowledge-graph with text chunks of the raw documents. This method is suitable for questions that require an understanding of specific entities mentioned in the documents (e.g. What are the healing properties of chamomile?).

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [2]:
INPUT_DIR = "/Volumes/Samsung-SSD-T7/Qredence/_Poetry/AgenticFleetPackage/GraphFleetProd/graphfleet/output/20240719-023135/artifacts"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [3]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

# load description embeddings to an in-memory lancedb vectorstore
# to connect to a remote db, specify url and port values.
description_embedding_store = LanceDBVectorStore(
    collection_name="entity_description_embeddings",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
entity_description_embeddings = store_entity_semantic_embeddings(
    entities=entities, vectorstore=description_embedding_store
)

print(f"Entity count: {len(entity_df)}")
entity_df.head()

Entity count: 142


,level,title,type,description,source_id,community,degree,human_readable_id,id,size,graph_embedding,top_level_node_id,x,y
0,0,"""DARREN EDGE""","""PERSON""","""Darren Edge is one of the authors of the pape...",76f53fb8722af58423d32a9f95018b18,2,2,0,b45241d70f0e43fca764df95b2b81f77,2.0,"[0.0011300550540909171, 0.0007336045382544398,...",b45241d70f0e43fca764df95b2b81f77,10.400290,-13.593511
1,0,"""HA TRINH""","""PERSON""","""Ha Trinh is one of the authors of the paper a...",76f53fb8722af58423d32a9f95018b18,2,2,1,4119fd06010c494caa07f439b333f4c5,2.0,"[0.001718277228064835, 0.0008772320579737425, ...",4119fd06010c494caa07f439b333f4c5,7.048254,1.012316
2,0,"""NEWMAN CHENG""","""PERSON""","""Newman Cheng is one of the authors of the pap...",76f53fb8722af58423d32a9f95018b18,1,2,2,d3835bf3dda84ead99deadbeac5d0d7d,2.0,"[0.0015600293409079313, 0.0006844564923085272,...",d3835bf3dda84ead99deadbeac5d0d7d,1.613804,6.890623
3,0,"""JOSHUA BRADLEY""","""PERSON""","""Joshua Bradley is one of the authors of the p...",76f53fb8722af58423d32a9f95018b18,1,2,3,077d2820ae1845bcbb1803379a3d1eae,2.0,"[0.0016834863927215338, 0.0007158922380767763,...",077d2820ae1845bcbb1803379a3d1eae,4.692405,7.699137
4,0,"""ALEX CHAO""","""PERSON""","""Alex Chao is one of the authors of the paper ...",76f53fb8722af58423d32a9f95018b18,0,2,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,2.0,"[0.0020669547375291586, 0.0004937680205330253,...",3671ea0dd4e84c1a9b02c5ab2c8f4bac,7.775792,0.226309


#### Read relationships

In [4]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

print(f"Relationship count: {len(relationship_df)}")
relationship_df.head()

Relationship count: 49


,source,target,weight,description,text_unit_ids,id,human_readable_id,source_degree,target_degree,rank
0,"""DARREN EDGE""","""GRAPH RAG""",1.0,"""Darren Edge is one of the contributors to the...",[76f53fb8722af58423d32a9f95018b18],eeef6ae5c464400c8755900b4f1ac37a,0,2,22,24
1,"""DARREN EDGE""","""MICROSOFT RESEARCH""",1.0,"""Darren Edge is affiliated with Microsoft Rese...",[76f53fb8722af58423d32a9f95018b18],422433aa45804c7ebb973b2fafce5da6,1,2,4,6
2,"""HA TRINH""","""GRAPH RAG""",1.0,"""Ha Trinh is one of the contributors to the de...",[76f53fb8722af58423d32a9f95018b18],86505bca739d4bccaaa1a8e0f3baffdc,2,2,22,24
3,"""HA TRINH""","""MICROSOFT RESEARCH""",1.0,"""Ha Trinh is affiliated with Microsoft Research.""",[76f53fb8722af58423d32a9f95018b18],1af9faf341e14a5bbf4ddc9080e8dc0b,3,2,4,6
4,"""NEWMAN CHENG""","""GRAPH RAG""",1.0,"""Newman Cheng is one of the contributors to th...",[76f53fb8722af58423d32a9f95018b18],353d91abc68648639d65a549e59b5cf3,4,2,22,24


In [5]:
covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")

claims = read_indexer_covariates(covariate_df)

print(f"Claim records: {len(claims)}")
covariates = {"claims": claims}

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Samsung-SSD-T7/Qredence/_Poetry/AgenticFleetPackage/GraphFleetProd/graphfleet/output/20240719-023135/artifacts/create_final_covariates.parquet'

#### Read community reports

In [6]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

print(f"Report records: {len(report_df)}")
report_df.head()

Report records: 5


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,0,# Graph RAG and Microsoft Research Community\n...,0,7.5,Graph RAG and Microsoft Research Community,The impact severity rating is high due to the ...,The community centers around the Graph RAG app...,[{'explanation': 'Graph RAG is a pivotal appro...,"{\n ""title"": ""Graph RAG and Microsoft Resea...",bd22085a-febd-46a6-9b6a-3c0839091f64
1,1,# Microsoft Strategic Missions and Technologie...,0,8.5,Microsoft Strategic Missions and Technologies ...,The impact severity rating is high due to the ...,The community centers around Microsoft Strateg...,[{'explanation': 'Microsoft Strategic Missions...,"{\n ""title"": ""Microsoft Strategic Missions ...",331db812-688b-457a-8bf8-cfac6be5eb25
2,2,# Microsoft Research and Graph RAG Development...,0,7.5,Microsoft Research and Graph RAG Development,The impact severity rating is high due to the ...,The community centers around Microsoft Researc...,[{'explanation': 'Microsoft Research is a pivo...,"{\n ""title"": ""Microsoft Research and Graph ...",5526e64f-ab93-4253-a31b-8a700a771790
3,3,# Naïve RAG and Evaluation Datasets\n\nThe com...,0,6.5,Naïve RAG and Evaluation Datasets,The impact severity rating is moderate to high...,"The community centers around Naïve RAG, a basi...",[{'explanation': 'Naïve RAG is a basic retriev...,"{\n ""title"": ""Na\u00efve RAG and Evaluation...",ccf704df-7bfb-41b4-9699-040b1159b819
4,4,# Graph RAG and Supporting Entities\n\nThe com...,0,8.5,Graph RAG and Supporting Entities,The impact severity rating is high due to the ...,The community revolves around the Graph RAG ap...,[{'explanation': 'Graph RAG is the central ent...,"{\n ""title"": ""Graph RAG and Supporting Enti...",9ea12a6b-abb9-47ff-b49c-6e911c3707da


#### Read text units

In [7]:
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)

print(f"Text unit records: {len(text_unit_df)}")
text_unit_df.head()

Text unit records: 5


,id,text,n_tokens,document_ids,entity_ids,relationship_ids
0,76f53fb8722af58423d32a9f95018b18,From Local to Global: A Graph RAG Approach to\...,3000,[0668cddc5f873265ba50da5a0a06edad],"[b45241d70f0e43fca764df95b2b81f77, 4119fd06010...","[eeef6ae5c464400c8755900b4f1ac37a, 422433aa458..."
1,4a924a483e795e5e40dbdb018113c06e,", 2010 and Jin et al., 2021). In our pipeline,...",3000,[0668cddc5f873265ba50da5a0a06edad],"[96aad7cb4b7d40e9b7e13b94a67af206, c9632a35146...","[a5e0d1644eb547ba9a5c3211aac4631a, 5a28b94bc63..."
2,77f9589bc5acd4c518314fae6c3769c8,media. It also includes specific examples of t...,3000,[0668cddc5f873265ba50da5a0a06edad],"[de988724cfdf45cebfba3b13c43ceede, dde131ab575...","[ce55841ebfdd47008bab8c258f10372e, 6090e736374..."
3,27f5bd73115aa334eae2c121c53f8a45,"2023), where retrieved event-plot sub-\ngraphs...",3000,[0668cddc5f873265ba50da5a0a06edad],"[de988724cfdf45cebfba3b13c43ceede, adf4ee3fbe9...","[38f51478f41f48db9bee570859b6f43e, 896d2a51e8d..."
4,39aecf1064d6b3e3cca60cd12e9ff74d,"Xiv:2303.08896 .\nMao, Y ., He, P., Liu, X., S...",1487,[0668cddc5f873265ba50da5a0a06edad],"[1943f245ee4243bdbfbd2fd619ae824a, c160b9cb27d...",None


In [8]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=None,
    api_type=OpenaiApiType.OpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)

### Create local search context builder

In [9]:
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    covariates=covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

NameError: name 'covariates' is not defined

### Create local search engine

In [ ]:
# text_unit_prop: proportion of context window dedicated to related text units
# community_prop: proportion of context window dedicated to community reports.
# The remaining proportion is dedicated to entities and relationships. Sum of text_unit_prop and community_prop should be <= 1
# conversation_history_max_turns: maximum number of turns to include in the conversation history.
# conversation_history_user_turns_only: if True, only include user queries in the conversation history.
# top_k_mapped_entities: number of related entities to retrieve from the entity description embedding store.
# top_k_relationships: control the number of out-of-network relationships to pull into the context window.
# include_entity_rank: if True, include the entity rank in the entity table in the context window. Default entity rank = node degree.
# include_relationship_weight: if True, include the relationship weight in the context window.
# include_community_rank: if True, include the community rank in the context window.
# return_candidate_context: if True, return a set of dataframes containing all candidate entity/relationship/covariate records that
# could be relevant. Note that not all of these records will be included in the context window. The "in_context" column in these
# dataframes indicates whether the record is included in the context window.
# max_tokens: maximum number of tokens to use for the context window.


local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

In [ ]:
search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

### Run local search on sample queries

In [ ]:
result = await search_engine.asearch("Tell me about Agent Mercer")
print(result.response)

In [ ]:
question = "Tell me about Dr. Jordan Hayes"
result = await search_engine.asearch(question)
print(result.response)

#### Inspecting the context data used to generate the response

In [ ]:
result.context_data["entities"].head()

In [ ]:
result.context_data["relationships"].head()

In [ ]:
result.context_data["reports"].head()

In [ ]:
result.context_data["sources"].head()

In [ ]:
if "claims" in result.context_data:
    print(result.context_data["claims"].head())

### Question Generation

This function takes a list of user queries and generates the next candidate questions.

In [ ]:
question_generator = LocalQuestionGen(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
)

In [ ]:
question_history = [
    "Tell me about Agent Mercer",
    "What happens in Dulce military base?",
]
candidate_questions = await question_generator.agenerate(
    question_history=question_history, context_data=None, question_count=5
)
print(candidate_questions.response)